In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from pathlib import Path
from tqdm import tqdm

import cv2
import pandas as pd
import numpy as np

In [2]:
path = Path('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data')
im_path = path/'images'
labels_path = path/'labels.csv'

### Load images' metadata

In [3]:
metadata = pd.read_csv(labels_path, delim_whitespace=True, header=None)
metadata.columns = ['im_path', 'quality', 'panel type']
metadata.quality = metadata["quality"].map(lambda value: 'bad' if value > 0 else 'good')
metadata['class'] = metadata[['panel type', 'quality']].agg(' '.join, axis=1)
metadata

,im_path,quality,panel type,class
0,images/cell0001.png,bad,mono,mono bad
1,images/cell0002.png,bad,mono,mono bad
2,images/cell0003.png,bad,mono,mono bad
3,images/cell0004.png,good,mono,mono good
4,images/cell0005.png,bad,mono,mono bad
...,...,...,...,...
2619,images/cell2620.png,good,poly,poly good
2620,images/cell2621.png,good,poly,poly good
2621,images/cell2622.png,good,poly,poly good
2622,images/cell2623.png,good,poly,poly good


## Image parameters

### Mean distribution

### Median distribution

### Std distribution

## Image embedding

In [4]:
model = foz.load_zoo_model("resnet50-imagenet-torch")
print(model.has_embeddings)

/home/khoa-ys/Projects/auto_inspection/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/khoa-ys/Projects/auto_inspection/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


True


In [5]:
image_list = list(im_path.iterdir())
print(len(image_list))
image_list[:10]

2624


[PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2346.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2391.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell0837.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell0625.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell0682.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2356.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2262.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2322.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2541.png'),
 

In [6]:
dataset = fo.load_dataset('dataset')
dataset.delete()

In [7]:
dataset = fo.Dataset('dataset')
for im_path in image_list:
    im_name = str('images/' + im_path.name)
    sample = fo.Sample(filepath=str(im_path))
    im_data = metadata[metadata['im_path'] == im_name]
    sample['label'] = fo.Classification(label=im_data['class'].values[0])
    dataset.add_sample(sample)

#dataset = fo.load_dataset('dataset')
dataset.persistent = True
dataset

Name:        dataset
Media type:  image
Num samples: 2624
Persistent:  True
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    label:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)

In [8]:
embeddings = dataset.compute_embeddings(model)

 100% |███████████████| 2624/2624 [23.8s elapsed, 0s remaining, 116.2 samples/s]      


In [9]:
results = fob.compute_visualization(
    dataset,
    embeddings=embeddings,
    num_dims=2,
    brain_key="image_embeddings",
    verbose=True,
    seed=51,
    method='umap'
)

Generating visualization...
UMAP(random_state=51, verbose=True)
Thu Sep  8 19:10:10 2022 Construct fuzzy simplicial set
Thu Sep  8 19:10:14 2022 Finding Nearest Neighbors
Thu Sep  8 19:10:16 2022 Finished Nearest Neighbor Search
Thu Sep  8 19:10:18 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Thu Sep  8 19:10:22 2022 Finished embedding


In [10]:
session = fo.launch_app(dataset)

In [11]:
labels = metadata['class']
plot = results.visualize(labels=labels)
plot.show(height=720)

FigureWidget({
    'data': [{'customdata': array(['631a13c539e5871246ef3e30', '631a13c539e5871246ef3e33',
    …